In [6]:
'''
Revised base on https://zhuanlan.zhihu.com/p/667766822
'''

# 创建DataLoader进行数据集转换
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler

device = 'cpu'
if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
    device = 'cuda'

class DataLoader:

    def __init__(self, batch_size, seq_len, pred_len):
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.target_slice = slice(0, None)

        self._read_data()

    # 读取数据
    def _read_data(self):
        '''
        扩展数据以缩短模型的训练时间至关重要；
        将缩放器安装在训练集上只是为了避免验证和测试集中的数据泄漏
        '''

        filepath = ('../Dataset/Simulate_Data/set_1/File_indexed.xlsx')

        df_raw = pd.read_excel(filepath)
        df = df_raw.set_index('TIME')

        # split train/valid/test
        n = len(df)
        train_end = int(n * 0.7)
        val_end = n - int(n * 0.2)
        test_end = n

        train_df = df[:train_end]
        val_df = df[train_end - self.seq_len : val_end]
        test_df = df[val_end - self.seq_len : test_end]

        # standardize by training set
        self.scaler = StandardScaler()
        self.scaler.fit(train_df.values)

        def scale_df(df, scaler):
            data = scaler.transform(df.values)
            return pd.DataFrame(data, index=df.index, columns=df.columns)

        self.train_df = scale_df(train_df, self.scaler)
        self.val_df = scale_df(val_df, self.scaler)
        self.test_df = scale_df(test_df, self.scaler)
        self.n_feature = self.train_df.shape[-1]

    # 将数据窗口分割为输入和标签
    def _split_window(self, data):
        inputs = data[:, : self.seq_len, :]
        labels = data[:, self.seq_len :, self.target_slice]

        inputs.set_shape([None, self.seq_len, None])
        labels.set_shape([None, self.pred_len, None])
        return inputs, labels

    def _make_dataset(self, data, shuffle=True):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=(self.seq_len + self.pred_len),
            sequence_stride=1,
            shuffle=shuffle,
            batch_size=self.batch_size,
        )
        ds = ds.map(self._split_window)
        return ds

    # DataLoader通过对预测进行逆变换，生成训练集、验证集和测试集
    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)

    def get_train(self, shuffle=True):
        return self._make_dataset(self.train_df, shuffle=shuffle)

    def get_val(self):
        return self._make_dataset(self.val_df, shuffle=False)

    def get_test(self):
        return self._make_dataset(self.test_df, shuffle=False)

ModuleNotFoundError: No module named 'tensorflow'

In [6]:
# 构建TSMixer网络

from tensorflow.keras import layers

def res_block(inputs, norm_type, activation, dropout, ff_dim):
    
    norm = layers.BatchNormalization

    # Time mixing
    '''
    混合器层，它包括：
        批量归一化
        转置矩阵
        通过 ReLu 激活馈送到全连接层
        再次转置
        漏失层
        并在最后添加残差
    '''
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feature mixing
    '''
    特征混合部分，其中：
        批量归一化
        致密层
        一个dropout层
        另一个致密层
        另一个dropout层
        并添加残差以进行残差连接
    '''
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(0.7)(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    return x + res

def build_model(
    input_shape,
    pred_len,
    n_block,
    ff_dim,
    target_slice,
):

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
        x = res_block(x, norm_type, activation, dropout, ff_dim)

    if target_slice:
        x = x[:, :, target_slice]

  # Temporal projection
    '''
    时间投影步骤：
        转置
        穿过致密层
        最终转置
    '''
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

In [13]:
# 初始化DataLoader来读取数据集并创建训练集、验证集和测试集
data_loader = DataLoader(batch_size=32, seq_len=512, pred_len=96)

train_data = data_loader.get_train()
val_data = data_loader.get_val()
test_data = data_loader.get_test()

# 初始化SMixer模型
model = build_model(
    input_shape=(512, data_loader.n_feature),
    pred_len=96,
    n_block=8,
    ff_dim=64,
    target_slice=data_loader.target_slice
)

# 训练模型
'''
使用学习率为 1e-4 的 Adam 优化器;
实施检查点来保存最佳模型;
在连续三个时期没有改进的情况下提前停止以停止训练。
'''
tf.keras.utils.set_random_seed(42)

optimizer = tf.keras.optimizers.Adam(1e-4)

model.compile(optimizer, loss='mse', metrics=['mae'])

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='tsmixer_checkpoints/',
    vebose=1,
    save_best_only=True,
    save_weights_only=True
)

early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3
)

history = model.fit(
    train_data,
    epochs= 30,
    validation_data=val_data,
    callbacks=[checkpoint_callback, early_stop_callback]
)

# 在模型训练完毕后加载检查点回调保存的最佳模型
best_epoch = np.argmin(history.history['val_loss'])

model.load_weights("tsmixer_checkpoints/")

# 访问96个时间步长的最后一个窗口的预测（已按比例缩放）
predictions = model.predict(test_data)

scaled_preds = predictions[-1,:,:]

# 将缩放和逆变换的预测存储在 DataFrame 中，以评估性能并稍后绘制预测
cols = ['HUFL', 'HULL', 'MUFL', 'MULL', 'LUFL', 'LULL', 'OT']

scaled_preds_df = pd.DataFrame(scaled_preds)
scaled_preds_df.columns = cols

preds = data_loader.inverse_transform(scaled_preds)

preds_df = pd.DataFrame(preds)
preds_df.columns = cols

AttributeError: module 'tensorflow_core.keras.preprocessing' has no attribute 'timeseries_dataset_from_array'

In [ ]:
# 可视化预测结果
tsmixer_preds = preds

cols_to_plot = ['HUFL', 'HULL', 'MUFL', 'MULL']

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,8))

for i, ax in enumerate(axes.flatten()):
    col = cols_to_plot[i]
    
    ax.plot(test_data['date'][-96:], test_data[col][-96:])
    ax.plot(test_data['date'][-96:], tsmixer_preds[col], label='TSMixer', ls='--', color='green')
    
    ax.legend(loc='best')
    ax.set_xlabel('Time steps')
    ax.set_ylabel('Value')
    ax.set_title(col)
    
plt.tight_layout()
fig.autofmt_xdate()